In [ ]:
#pip install requests

In [ ]:

import socket
import time
import pylsl
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
%matplotlib qt
#%matplotlib inline

# SELECT DATA TO STREAM
acc = True      # 3-axis acceleration
bvp = True      # Blood Volume Pulse
gsr = True      # Galvanic Skin Response (Electrodermal Activity)
tmp = True      # Temperature
ibi = True      # Interbeat Interval and Heartbeat
bat = True      # Device Battery
tag = True      # Tag taken from the device (by pressing the button)

serverAddress = '127.0.0.1'
serverPort = 28000
bufferSize = 4096

deviceID = 'CDCB11' # 'A03003'

def connect():
    global s
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(3)

    print("Connecting to server")
    s.connect((serverAddress, serverPort))
    print("Connected to server\n")

    print("Devices available:")
    s.send("device_list\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

    print("Connecting to device")
    s.send(("device_connect " + deviceID + "\r\n").encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))

    print("Pausing data receiving")
    s.send("pause ON\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))
connect()

time.sleep(1)

def suscribe_to_data():
    if acc:
        print("Suscribing to ACC")
        s.send(("device_subscribe " + 'acc' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if bvp:
        print("Suscribing to BVP")
        s.send(("device_subscribe " + 'bvp' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if gsr:
        print("Suscribing to GSR")
        s.send(("device_subscribe " + 'gsr' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if tmp:
        print("Suscribing to Temp")
        s.send(("device_subscribe " + 'tmp' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if ibi:
        print("Suscribing to IBI")
        s.send(("device_subscribe " + 'ibi' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if bat:
        print("Suscribing to Batt")
        s.send(("device_subscribe " + 'bat' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))
    if tag:
        print("Suscribing to Tag")
        s.send(("device_subscribe " + 'tag' + " ON\r\n").encode())
        response = s.recv(bufferSize)
        print(response.decode("utf-8"))

    print("Resuming data receiving")
    s.send("pause OFF\r\n".encode())
    response = s.recv(bufferSize)
    print(response.decode("utf-8"))
suscribe_to_data()

time.sleep(1)

def reconnect():
    print("Reconnecting...")
    connect()
    suscribe_to_data()
    #stream()

def disconnect():
    s.send("device_disconnect\r\n".encode())

#global bvp_timestamp
bvp_timestamp = []
#global bvp_data
bvp_data = []
temp_timestamp = []
temp_data = []
acc_timestamp = []
acc_data = []
gsr_timestamp = []
gsr_data = []
#ibi_timestamp = []
#ibi_data = []


while True:
    response = s.recv(bufferSize).decode("utf-8")
    #print(response)
    if "connection lost to device" in response:
        print(response.decode("utf-8"))
        reconnect()
    samples = response.split("\n")
    for i in range(len(samples)-1):
        stream_type = samples[i].split()[0]
        if stream_type == "E4_Bvp":
            bvp_new_timestamp = float(samples[i].split()[1].replace(',','.'))
            bvp_new_data = float(samples[i].split()[2].replace(',','.'))
            bvp_timestamp.append(bvp_new_timestamp)
            bvp_data.append(bvp_new_data)                       
        if stream_type == "E4_Temperature":
            temp_new_timestamp = float(samples[i].split()[1].replace(',','.'))
            temp_new_data = float(samples[i].split()[2].replace(',','.'))
            temp_timestamp.append(temp_new_timestamp)
            temp_data.append(temp_new_data)
        if stream_type == "E4_Acc":
            timestamp = float(samples[i].split()[1].replace(',','.'))
            data = [int(samples[i].split()[2].replace(',','.')), int(samples[i].split()[3].replace(',','.')), int(samples[i].split()[4].replace(',','.'))]
            acc_timestamp.append(timestamp)
            acc_data.append(data)
        if stream_type == "E4_Gsr":
            timestamp = float(samples[i].split()[1].replace(',','.'))
            data = float(samples[i].split()[2].replace(',','.'))
            gsr_timestamp.append(timestamp)
            gsr_data.append(data)
        if stream_type == "E4_Ibi":
            timestamp = float(samples[i].split()[1].replace(',','.'))
            data = float(samples[i].split()[2].replace(',','.'))
            #ibi_timestamp.append(timestamp)
            #ibi_data.append(data)
            outletIBI.push_sample([data], timestamp=timestamp)
        if stream_type == "E4_Tag":
            timestamp = float(samples[i].split()[1].replace(',','.'))
            data = float(samples[i].split()[2].replace(',','.'))
            outletTag.push_sample([data], timestamp=timestamp)
        if stream_type == "E4_Battery":
            timestamp = float(samples[i].split()[1].replace(',','.'))
            data = float(samples[i].split()[2].replace(',','.'))
            outletBatt.push_sample([data], timestamp=timestamp)


In [ ]:
### Works to send data to PowerBI at specified address for a dataset


import requests
import json
from datetime import datetime 
from random import randint
import time

myurl = "https://api.powerbi.com/beta/5ee35505-eb8e-4929-937d-645df5013288/datasets/50ad263b-54e0-44ba-93e7-b00cbc39f84d/rows?key=vO%2BdZoHdf6qaU5fg86ZXyj8H%2B4sGYbjCaU6CIvKh8kdC7hnu24Rp38n7785Cvjr4ldnLMGUjL9WuMsZH0CfDsw%3D%3D"

for i in range(10):
    print(i)
    date = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    data= [
    {
    "temp_datetime" :date,
    "temp_data" :randint(0,100),
    "gsr_datetime" :date,
    "gsr_data" :randint(25,200)
    }
    ]

    r = requests.post(myurl, json.dumps(data))
    print(r.status_code)
    time.sleep(2)

In [60]:
### Works to send data to PowerBI at specified address for a dataset


import requests
import json
from datetime import datetime 
from random import randint
import time

myurl = "https://api.powerbi.com/beta/5ee35505-eb8e-4929-937d-645df5013288/datasets/50ad263b-54e0-44ba-93e7-b00cbc39f84d/rows?key=vO%2BdZoHdf6qaU5fg86ZXyj8H%2B4sGYbjCaU6CIvKh8kdC7hnu24Rp38n7785Cvjr4ldnLMGUjL9WuMsZH0CfDsw%3D%3D"


date = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
data= [
{
"temp_datetime" :date,
"temp_data" :60,
"gsr_datetime" :date,
"gsr_data" :50
}, 
{
"temp_datetime" :date,
"temp_data" :70,
"gsr_datetime" :date,
"gsr_data" :60
}
]

r = requests.post(myurl, json.dumps(data))
print(r.status_code)


200


In [14]:
data= [{'temp_data' : (100, 200)}]

print(json.dumps(data))

[{"temp_data": [100, 200]}]


In [22]:
from datetime import datetime 

print(datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))

2021-07-19T13:21:02
